In [112]:
#!pip install langchain
#!pip install langchain-openai
#!pip install langgraph
#!pip install grandalf

# Diagrama de Secuencia para explicacion de como funciona el Agente reflexivo

```mermaid
sequenceDiagram
    User ->>+ Agent: Resuelve este Ejercicio
    Agent ->>+ Agent: Lee el API Key de OpenAI
    Agent ->> GenerativeAgent: Configura el Prompt de inicial de sistema para el nodo generativo
    Agent ->> ReflexiveAgent : Configura el Prompt de inicial de sistema para el nodo reflexivo
    Agent ->> GenerativeAgent: Start to solve this
    GenerativeAgent ->>+ ReflexiveAgent: Result
    ReflexiveAgent ->>+ GenerativeAgent: Make this Fixes
    GenerativeAgent ->>+ ReflexiveAgent: Improve Result
    ReflexiveAgent ->>+ GenerativeAgent: Is good Enought
    GenerativeAgent ->> Agent : Process Finished
    Agent ->> Agent: Archivo .MD + code_file py or pl
    Agent ->> Agent: Excecute the Code , print the response
    Agent ->> User : Done
```

In [113]:
import json

api_key = None
with open('config.json') as f:
    data = json.load(f)
    api_key = data['openai']['api_key']

In [114]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# Generation Prompt Template
generation_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
You are an expert in Prolog and functional Python.

Your task is to:
1. Analyze the problem statement.
2. Provide a concise and correct solution using ONLY the language requested (Prolog or Python).
3. Always include a complete, executable code block.
4. Provide examples of how to use or run the code.

Respond in the following JSON format:
{{
  "explanation": "A clear explanation of your reasoning and approach.",
  "code": "The complete code solution, ready to run."
}}

DO NOT mix languages in your response.
"""
    ),
    MessagesPlaceholder(variable_name="messages"),
])

# Reflection Prompt Template
reflection_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
You are a reflective agent whose task is to critique and improve previous responses.

1. Review the last answer with a critical and constructive mindset.
2. Identify any of the following:
   - Logical flaws or inefficiencies
   - Areas lacking clarity or beginner-friendliness
   - Violations of idiomatic usage or conventions in Prolog/Python
   - Missing parts of the original task or prompt
3. If the answer is already strong, explain why and confirm it.
4. If improvements are needed, revise the explanation and/or code, and output the updated version.

Respond in the following JSON format:
{{
  "reflection": "Your critical analysis and reasoning.",
  "suggested_improvement": {{
     "explanation": "Improved explanation (if needed)",
     "code": "Revised complete code (if needed)"
  }}
}}
"""
    ),
    MessagesPlaceholder(variable_name="messages"),
])

llm = ChatOpenAI(api_key=api_key, model="gpt-4o", temperature=0.3)

generate_chain = generation_prompt | llm
reflect_chain = reflection_prompt | llm


In [115]:
from typing import List, Sequence
import json
import re
from dotenv import load_dotenv

from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, MessageGraph

load_dotenv()

REFLECT = "reflect"
GENERATE = "generate"

def generation_node(state: Sequence[BaseMessage]):
    """Generates an initial or revised solution based on current state."""
    return generate_chain.invoke({"messages": state})

def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    """Critiques the generated message and returns the reflection as a new HumanMessage."""
    res = reflect_chain.invoke({"messages": messages})
    return [HumanMessage(content=res.content)]  

builder = MessageGraph()
builder.add_node(GENERATE, generation_node)
builder.add_node(REFLECT, reflection_node)

builder.set_entry_point(GENERATE)

def should_continue(state: List[BaseMessage]) -> str:
    """
    Verifica si la reflexión sugiere mejoras. Si no, se considera suficiente y termina el grafo.
    """
    if len(state) >= 6:
        return END

    last_message = state[-1].content.strip()

    cleaned = re.sub(r"^```(?:json)?\n", "", last_message)
    cleaned = re.sub(r"\n```$", "", cleaned)

    try:
        parsed = json.loads(cleaned)
        if "suggested_improvement" in parsed:
            
            return REFLECT
        else:
            print("No suggested improvements : ---------------------------------------------------")
            print(parsed)
            return END
    except json.JSONDecodeError:
        return REFLECT


builder.add_conditional_edges(GENERATE, should_continue)
builder.add_edge(REFLECT, GENERATE)

graph = builder.compile()

print(graph.get_graph().draw_mermaid())
graph.get_graph().print_ascii()


%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	generate(generate)
	reflect(reflect)
	__end__([<p>__end__</p>]):::last
	__start__ --> generate;
	reflect --> generate;
	generate -.-> reflect;
	generate -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

          +-----------+            
          | __start__ |            
          +-----------+            
                *                  
                *                  
                *                  
          +----------+             
          | generate |             
          +----------+             
          ...        ...           
         .              .          
       ..                ..        
+---------+           +---------+  
| reflect |           | __end__ |  
+---------+           +---------+  


In [116]:
import json
import re
from langchain_core.messages import HumanMessage, AIMessage

def convert_to_markdown(conversation):
    md_output = "# 🧠 **Inteligencia Artificial - Conversación**\n\n"

    for i, msg in enumerate(conversation):
        role = "Human" if isinstance(msg, HumanMessage) else "AI"
        header = f"## {i + 1}️⃣ {role} Message\n"
        content = msg.content.strip()
        md_output += header

        if role == "Human":
            # Intenta detectar si el contenido es un JSON con reflexión
            cleaned = re.sub(r"^```(?:json)?\n", "", content)
            cleaned = re.sub(r"\n```$", "", cleaned)

            try:
                parsed = json.loads(cleaned)

                # Si es una reflexión con mejora sugerida
                reflection = parsed.get("reflection", "").strip()
                suggestion = parsed.get("suggested_improvement", {})

                explanation = suggestion.get("explanation", "").strip()
                code = suggestion.get("code", "").strip()

                if reflection:
                    md_output += f"\n**🪞 Reflexión del agente:**\n\n{reflection}\n"
                if explanation:
                    md_output += f"\n**🧠 Explicación sugerida:**\n\n{explanation}\n"
                if code:
                    lang = "prolog" if ":-" in code or "writeln" in code else "python"
                    md_output += f"\n**💻 Código sugerido:**\n\n```{lang}\n{code}\n```\n"

            except json.JSONDecodeError:
                # Si no es un JSON válido, solo lo imprime como está
                md_output += f"\n**🧑 Entrada del usuario:**\n\n```\n{content}\n```\n\n"

        else:
            # Procesamiento de respuestas del AI (formato json con explanation y code)
            cleaned = re.sub(r"^```(?:json)?\n", "", content)
            cleaned = re.sub(r"\n```$", "", cleaned)

            try:
                parsed = json.loads(cleaned)

                explanation = parsed.get("explanation", "").strip()
                code = parsed.get("code", "").strip()

                if explanation:
                    md_output += f"\n**🧠 Explicación:**\n\n{explanation}\n"
                if code:
                    lang = "prolog" if ":-" in code or "writeln" in code else "python"
                    md_output += f"\n**💻 Código:**\n\n```{lang}\n{code}\n```\n"

            except json.JSONDecodeError as e:
                md_output += f"\n⚠️ Error al analizar JSON:\n```\n{e}\n```\n"
                md_output += f"\n🔍 Contenido bruto:\n```\n{content}\n```\n"

    return md_output


In [117]:
def test_code_file(filepath: str):
    import subprocess

    if filepath.endswith(".py"):
        print(f"🧪 Probando archivo Python: {filepath}")
        try:
            result = subprocess.run(["python3", filepath], capture_output=True, text=True, timeout=10)
            if result.returncode == 0:
                print("✅ Python ejecutado correctamente.")
                print(result.stdout)
            else:
                print("❌ Error en el código Python:")
                print(result.stderr)
        except Exception as e:
            print(f"❌ Excepción al ejecutar Python: {e}")

    elif filepath.endswith(".pl"):
        print(f"🧪 Probando archivo Prolog: {filepath}")
        try:
            result = subprocess.run(["swipl", "-q", "-f", filepath, "-g", "main", "-t", "halt"], capture_output=True, text=True, timeout=10)
            if result.returncode == 0:
                print("✅ Prolog ejecutado correctamente.")
                print(result.stdout)
            else:
                print("❌ Error en el código Prolog:")
                print(result.stderr)
        except Exception as e:
            print(f"❌ Excepción al ejecutar Prolog: {e}")
    else:
        print(f"⚠️ Tipo de archivo no soportado para prueba: {filepath}")


In [118]:
import os

def save_code_if_final(messages: List[BaseMessage], index: int):
    last = messages[-1].content.strip()
    cleaned = re.sub(r"^```(?:json)?\n", "", last)
    cleaned = re.sub(r"\n```$", "", cleaned)

    try:
        parsed = json.loads(cleaned)

        if "suggested_improvement" in parsed:
            return

        code = parsed.get("code", "").strip()
        if not code:
            return

        lang = "prolog" if ":-" in code or "writeln" in code else "python"
        extension = ".pl" if lang == "prolog" else ".py"
        filename = f"outputs/ejercicio{index}{extension}" if lang == "prolog" else f"outputs/solucion{index}{extension}"

        os.makedirs("outputs", exist_ok=True)
        with open(filename, "w", encoding="utf-8") as f:
            f.write(code)

        print(f"💾 Código final guardado en: {filename}")

        test_code_file(filename)

    except json.JSONDecodeError as e:
        print(f"⚠️ No se pudo guardar el código final del ejercicio {index} (JSON inválido)")


In [119]:
import os
import json
import re
from langchain_core.messages import HumanMessage, AIMessage
 

def print_markdown(text, filename):
    # Ensure output folder exists
    os.makedirs("outputs", exist_ok=True)

    # Save to outputs/filename
    path = os.path.join("outputs", filename)
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)

def read_file_contents(file_path: str) -> str:
    if not os.path.exists(file_path):
        return f"⚠️ El archivo '{file_path}' no fue encontrado.\n"
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()
    return f"\n📄 Contenido del archivo `{os.path.basename(file_path)}`:\n\n```\n{content}\n```\n"


problems = [
   """En clase compartimos unos programas lógicos de ejemplo, que demostraban el poder del
proceso de inferencia del lenguaje utilizando backtracking. El backtracking es una
herramienta poderosa tanto en lenguajes lógicos como en lenguajes funcionales. En
particular, los lenguajes funcionales carecen de iteradores (lazos for) y solo pueden recorrer
estructuras de datos a través de llamadas recursivas. En particular, hay dos funciones
clásicas a implementar en todo lenguaje funcional: fold y map. La función fold se encarga de
recibir una lista (generalmente) y aplicar una función que recibe como parámetro para
operar todos los elementos de la lista y obtener un único resultado (consolidar), de forma
que si uno llama a la función fold (+) [1, 2, 3, 4], es decir, usar el operador más en los
elementos de la lista, el resultado es la sumatoria de los elementos de la lista, es decir: 10.
En el caso de map, esta es una función que permite ejecutar una función tomando como
argumento cada elemento de una lista, y genera una nueva lista con el resultado de aplicar
esta función en cada elemento, así: map (+1) [1, 2, 3, 4], retorna [2, 3, 4, 5].
En este ejercicio, implemente una versión de map y fold en PROLOG, en particular para el
cálculo de la sumatoria de una lista (para fold) y (+1) para map.
"""
,
"""
Dado el archivo familias.pl, que consiste en una base de conocimiento de relaciones
familiares y los predicados que permiten inferir relaciones válidas entre los miembros de
familia, cargue la base de conocimiento, revise la validez de los predicados, complete los
predicados faltantes y realice 15 preguntas que sean de su interés de la base de
conocimiento. Registre sus resultados en un único documento. este es el contenido de el archivo familias.pl:""" + read_file_contents("archivos_adjuntos/familias.pl")
,
"""
Dado el archivo fibo_errors.pl, que contiene una implementación del cálculo del enésimo
número de la serie de Fibonacci. Revise la implementación y repare el código para que
genere correctamente el enésimo número de la serie y registre sus resultados en el
documento de entrega.
""" + read_file_contents("archivos_adjuntos/fibo_errors.pl")
,
    """
Dado el archivo exshell-cars.zip, que contiene una implementación de un sistema experto
escrito en PROLOG, cree un nuevo sistema experto agergando 10 reglas nuevas que le
permita evaluar su nuevo sistema experto. Registre sus resultados en el documento de
entrega.
    """]


for i, problem in enumerate(problems):
    print(f"🚀 Resolviendo problema {i + 1}...")

    input_message = HumanMessage(content=problem)
    response_messages = graph.invoke(input_message)
    markdown_text = convert_to_markdown(response_messages)

    filename = f"Solution{i + 1}.md"
    print_markdown(markdown_text, filename)
    save_code_if_final(response_messages, i)

    print(f"✅ Problema {i + 1} guardado como '{filename}'\n")

🚀 Resolviendo problema 1...
No suggested improvements : ---------------------------------------------------
{'explanation': 'In Prolog, we can implement the `map` and `fold` functions using recursion. For `map`, we define a predicate `map_increment/2` that takes a list and returns a new list where each element is incremented by 1. We use recursion to process each element of the list, increment it, and construct the new list. For `fold`, we define a predicate `fold_sum/2` that takes a list and returns the sum of its elements. Again, we use recursion to traverse the list, accumulating the sum as we go. Both predicates handle the base case of an empty list, where the result is an empty list for `map` and 0 for `fold`.', 'code': 'map_increment([], []).\nmap_increment([H|T], [H1|T1]) :-\n    H1 is H + 1,\n    map_increment(T, T1).\n\nfold_sum([], 0).\nfold_sum([H|T], Sum) :-\n    fold_sum(T, RestSum),\n    Sum is H + RestSum.\n\n% Example usage:\n% ?- map_increment([1, 2, 3, 4], Result).\n%